# Sudan States, South Sudan States Sorting

Baylasan Innovation, Sudan,

مخيم الوهمة,

Author(s): Rami Ahmed (KPI)

## Imports

In [1]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
import plotly.graph_objects as go

### Open plot in another tab

In [3]:
import plotly.io as pio
pio.renderers.default = 'firefox'

## Load Geo Data

In [4]:
sudan = json.load(open("SD_SSD_States.geojson", "r"))

In [5]:
state_id_map = {}
for feature in sudan["features"]:
    feature["id"] = feature["properties"]["StateNameAr"]
    state_id_map[feature["properties"]["StateID"]] = feature["id"]

## Load Data

### Static Data

In [6]:
GSID = '13ot0rYLxegd-JxNBXjcF1obxwPzhl99sKSGICJpINiY'
WSN = 'StatesPoints'
URL = f'https://docs.google.com/spreadsheets/d/{GSID}/gviz/tq?tqx=out:csv&sheet={WSN}'

In [7]:
df = pd.read_csv(URL)

### Animated Data

In [ ]:
GSID = '13ot0rYLxegd-JxNBXjcF1obxwPzhl99sKSGICJpINiY'
WSN = 'StatesPointsTime'
URL = f'https://docs.google.com/spreadsheets/d/{GSID}/gviz/tq?tqx=out:csv&sheet={WSN}'

In [ ]:
df = pd.read_csv(URL)

### Attatch an ID Column from 'geojson file' Into the DF w/ State's Name

In [8]:
df['ID'] = df["الولاية"].apply(lambda x: list(state_id_map.keys())[list(state_id_map.values()).index(x)])

### Create Separate Data for Sudan/South Sudan

In [9]:
dfsd = df.copy()

In [10]:
# Clear South Sudan's ID's:
for i in range(0,len(df)):
    if df[i:i+1]['ID'][i].startswith('SS'):
        dfsd['الولاية'][i:i+1] = ''

/home/sketcher/anaconda3/envs/baylasan/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [11]:
dfss = df.copy()

In [12]:
# Clear Sudan's ID's:
for i in range(0,len(df)):
    if df[i:i+1]['ID'][i].startswith('SS') is False:
        dfss['الولاية'][i:i+1] = ''

/home/sketcher/anaconda3/envs/baylasan/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## Plotting

### 2D Plots

In [ ]:
# df["النقاط"].plot()

### Choropleth Plot

#### Static

##### Sudan

In [13]:
######################
fig = px.choropleth(
    dfsd,
    locations='الولاية', # From DF --> GeoJson id map
    geojson=sudan,
    color="النقاط",
    title="ترتيب الولايات حسب الجاهزية",
    color_continuous_scale=px.colors.diverging.RdYlGn,
).update_geos(fitbounds="locations", visible=False).update_traces(hovertemplate=None).update_layout(
font={"color": "rgb(64,64,64)",
      "family": "Times New Roman",
      "size": 20},
titlefont={"color": "rgb(64,64,64)",
           "family": "Times New Roman",
           "size": 40}
)

fig.show()
######################

In [14]:
# Create Data for Top 3 States
dfsd_top = dfsd.copy()
# Clear points less than top 3:
for i in range(0,len(dfsd)):
    if dfsd[i:i+1]["النقاط"][i] < dfsd.nlargest(3, "النقاط")["النقاط"].min():
        dfsd_top = dfsd_top.drop([i])

In [15]:
dfsd_top = dfsd_top.sort_values('النقاط',ascending=False)

In [16]:
# Star Top 3 States:
fig.add_scattergeo(
    locations=dfsd_top["الولاية"],
    geojson=sudan,
    text=dfsd_top["الولاية"],
    textfont = {"color": "rgb(32,32,32)",
                "family": "Arial, Bold",
                "size": 20},
    textposition="bottom center",
    marker= dict(size=[30,25,20],
                 color=['Gold', 'Silver', 'Brown'],
                 opacity=1,
                 symbol='star'
                ),
    mode ="markers+text",
    showlegend=False,
    hoverinfo='skip'
)

##### Add South Sudan

In [17]:
# Make South Sudan Points Nuetral
for i in range(0,len(df)):
    if dfss[i:i+1]['ID'][i].startswith('SS') is True:
        dfss["النقاط"][i:i+1] = (df["النقاط"].min() + df["النقاط"].max())/2

/home/sketcher/anaconda3/envs/baylasan/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [18]:
fig.add_choropleth(
    autocolorscale=False,
    z=dfss["النقاط"],
    locations=dfss["الولاية"],
    locationmode='geojson-id',
    geojson=sudan,
    showscale=False,
    marker=dict(opacity=0.5),
    hoverinfo='location'
).update_traces(name='')

In [ ]:
fig.write_html("SD_SSD_States_Static.html")

#### Animation

##### Sudan

In [ ]:
######################
fig = px.choropleth(
    dfsd,
    animation_frame='Time',
    locations='الولاية',
    geojson=sudan,
    color="النقاط",
    title="ترتيب الولايات حسب الجاهزية",
    color_continuous_scale=px.colors.diverging.RdYlGn
).update_geos(fitbounds="locations", visible=False).update_traces(hovertemplate=None).update_layout(
font={"color": "rgb(64,64,64)",
      "family": "Times New Roman",
      "size": 20},
titlefont={"color": "rgb(64,64,64)",
           "family": "Times New Roman",
           "size": 40}
)

fig.show()
######################

##### Add South Sudan

In [ ]:
# Make South Sudan Points Nuetral
for i in range(0,len(df)):
    if dfss[i:i+1]['ID'][i].startswith('SS') is True:
        dfss["النقاط"][i:i+1] = (df["النقاط"].min() + df["النقاط"].max())/2

In [ ]:
fig.add_choropleth(
    autocolorscale=False,
    z=dfss["النقاط"],
    locations=dfss["الولاية"],
    locationmode='geojson-id',
    geojson=sudan,
    showscale=False,
    marker=dict(opacity=0.5),
    hoverinfo='location'
).update_traces(name='')

In [ ]:
fig.write_html("SD_SSD_States_Animation.html")

# --------------------------------------------------------------------------------------------------